In [1]:
!nvidia-smi

Mon Aug 29 13:16:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:17:00.0 Off |                  N/A |
|  0%   39C    P8    16W / 350W |   1481MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:65:00.0 Off |                  N/A |
| 30%   

In [2]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict
#from UV_Encoders import UV_Encoder
#from UV_Aggregators import UV_Aggregator
#from Social_Encoders import Social_Encoder
#from Social_Aggregators import Social_Aggregator
import torch.nn.functional as F
import torch.utils.data
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os



In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

embed_dim = 128
dir_data = './data/toy_dataset'

path_data = dir_data + ".pickle"
data_file = open(path_data, 'rb')
history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(
    data_file)

In [4]:
trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                              torch.FloatTensor(train_r))
testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                         torch.FloatTensor(test_r))

In [5]:
batch_size = 128
test_batch_size = 128

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)

# 705명 유저
num_users = history_u_lists.__len__()

# 1941개 아이템 수 
num_items = history_v_lists.__len__()

# 8등급, 2.0: 0, 1.0: 1, 3.0: 2, 4.0: 3, 2.5: 4, 3.5: 5, 1.5: 6, 0.5: 7
num_ratings = ratings_list.__len__()

In [6]:
# embed_dim = 128차원 

u2e = nn.Embedding(num_users, embed_dim).to(device)
v2e = nn.Embedding(num_items, embed_dim).to(device)
r2e = nn.Embedding(num_ratings, embed_dim).to(device)

In [7]:
u2e, v2e, r2e

(Embedding(705, 128), Embedding(1941, 128), Embedding(8, 128))

### 처음 시작하는 부분 노드 128개 시작 

In [8]:
running_loss = 0.0
for i, data in enumerate(train_loader, 0):
    batch_nodes_u, batch_nodes_v, labels_list = data
    #optimizer.zero_grad()
    break


In [9]:
batch_nodes_u, batch_nodes_v, labels_list

(tensor([595, 327, 318, 699, 139, 689,  43, 231, 110, 375, 637,  85, 447,  23,
         256, 380, 509, 515, 614, 300,  39, 626, 642, 104, 681, 316,  76, 496,
         382, 498,  23, 336, 490, 673, 688, 674, 211, 665,  38, 428, 458, 579,
         296, 546, 225, 296, 192, 262, 315, 225, 257, 225,  80,  75, 216, 225,
         461, 607, 276, 499, 408, 180, 360, 392, 288,  83, 196, 459, 425, 447,
         143,  82, 198,  88, 124, 193, 463, 440,   8, 428, 321, 514, 503, 459,
         216, 398,  53, 377, 666, 533,   9, 430,  58, 259, 689, 116, 155,  99,
         183, 324, 147, 703, 509, 228,  58, 451, 262,  53, 444, 374, 424, 296,
         267, 620,   2, 573, 248, 325,  74,  88, 258, 637, 625,  53, 146, 700,
         508, 493]),
 tensor([  68,   74,   86,   74,   31,   76,  661,  132,   26,   47, 1472,   32,
           78,  726, 1526,  673,    6,   90,  120,   72,   73, 1282,   68,   72,
          267,   32,   17,    0,   86,   34,  383,  120,  333,   39,   11,  102,
          296,   47,  997

### UV_Encoder 들어가는 부분 

In [10]:
node = 128

In [11]:
tmp_history_uv = []
tmp_history_r = []
for node in batch_nodes_u:
    tmp_history_uv.append(history_u_lists[int(node)])
    tmp_history_r.append(history_ur_lists[int(node)])

len(tmp_history_uv), len(tmp_history_r)

(128, 128)

In [29]:
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
import random
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self, embedding_dims):
        super(Attention, self).__init__()
        self.embed_dim = embedding_dims
        self.bilinear = nn.Bilinear(self.embed_dim, self.embed_dim, 1)
        self.att1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.att2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.att3 = nn.Linear(self.embed_dim, 1)
        self.softmax = nn.Softmax(0)

    def forward(self, node1, u_rep, num_neighs):
        uv_reps = u_rep.repeat(num_neighs, 1)
        x = torch.cat((node1, uv_reps), 1)
        x = F.relu(self.att1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.att2(x))
        x = F.dropout(x, training=self.training)
        x = self.att3(x)
        att = F.softmax(x, dim=0)
        return att

In [66]:
uv = True
# neigh_feats = self.aggregator.forward(nodes, tmp_history_uv, tmp_history_r)  # user-item network 
# 위에 부분을 사용한 것. 
'''

neigh_feats = agg_u_history.forward(batch_nodes_u, tmp_history_uv, tmp_history_r)

'''

embed_dim = 128
device = 'cpu'

# 초기화 해주기 위한 빈 torch 
embed_matrix = torch.empty(len(tmp_history_uv), embed_dim, dtype=torch.float).to(device)
#print(embed_matrix.shape)

w_r1 = nn.Linear(embed_dim * 2, embed_dim).to(device)
w_r2 = nn.Linear(embed_dim, embed_dim).to(device)
#att = Attention(embed_dim)

att = Attention(embed_dim)

for i in range(len(tmp_history_uv)): 
    history = tmp_history_uv[i]
    num_histroy_item = len(history) # i = 0일때, 10개 
    tmp_label = tmp_history_r[i]
    
    if uv == True:
        # user component
        e_uv = v2e.weight[history].to(device)
        uv_rep = u2e.weight[batch_nodes_u[i]].to(device) # i = 0일때, torch.Size([128])
    else:
        # item component
        e_uv = u2e.weight[history].to(deivce)
        uv_rep = v2e.weight[batch_nodes_u[i]].to(device)

    e_r = r2e.weight[tmp_label].to(device)
    x = torch.cat((e_uv, e_r), 1)
    #x = x.to('cpu') ## 제거 가능 
    
    x = F.relu(w_r1(x))
    o_history = F.relu(w_r2(x)).to(device) # i = 0일때, torch.Size([10, 128])

    # attention 함수 실행 부분 
    att_w = att(o_history, uv_rep, num_histroy_item) # i = 0일때, (torch.Size([10, 128]), torch.Size([128]), 10) ---> torch.Size([10, 1])
    att_history = torch.mm(o_history.t(), att_w) # torch.Size([128, 1])
    att_history = att_history.t() # torch.Size([1, 128])

    embed_matrix[i] = att_history # torch.Size([128, 128])
    
to_feats = embed_matrix

torch.Size([128, 128])


In [73]:
linear1 = nn.Linear(2 * embed_dim, embed_dim) 
neigh_feats = to_feats.to(device)

self_feats = u2e.weight[batch_nodes_u].to(device)

combined = torch.cat([self_feats, neigh_feats], dim=1)
combined = F.relu(linear1(combined)) # torch.Size([128, 128])

In [76]:
combined

tensor([[0.0000, 0.1891, 0.0242,  ..., 0.3887, 0.1054, 0.0823],
        [0.0000, 0.0206, 0.0000,  ..., 0.0993, 0.0000, 0.0134],
        [0.0000, 0.3735, 0.0000,  ..., 0.2190, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.2187,  ..., 0.8151, 0.0480, 0.0332],
        [0.6975, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2481],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.3918, 0.0000]],
       grad_fn=<ReluBackward0>)